transfer learning with keras

## encoding=utf-8
# 从某一层开始获取权重并开始transferlearning
# 输入图像的大小建议和pre_process model要求的输入大小一致，效果会更好
# 从处理方式的灵活性上推荐函数式模型，sequential只是特例
# 各层的激活函数优先选sigmoid ，即便是最后一层的softmax?
# 将最后一层的输出作为输入的ＴＬ实验３ 起始准确率不到２０　而本实验直接将池化层的结果作为输入　然后ｓｏｆｔｍａｘｘ获得的起始准确率达６０％

#总结一下：　ｔransfer learning :利用现有的模型，将特征层剥离，然后fine-truning
# １可以直接将输出层作为输入
# ２将中间某层（最好是特征层，比如卷积后，ＦＣ已经到达了抽象的感知事物的理解程度）或多层（trainable）
# 另外，还有就是自己写将多次结果的数据写到ｈ５文件中，然后读取

# keras 2.1.5
# 报错：StopIteration: unsupported operand type(s) for /=: 'JpegImageFile' and 'float'
# 降级到2.1.3

In [4]:
%matplotlib inline
from keras.models import *
from keras.layers import *
from keras.applications import *
from keras.preprocessing.image import ImageDataGenerator

os.environ["CUDA_VISIBLE_DEVICES"] = "0"
#释放显存
os.system("nvidia-smi | grep vpython3 | awk '{print $3}' | xargs kill -9")

0

In [5]:
height=224
train_batch_size=64
test_batch_size=64
FLAG = None
#assert

In [6]:
train_data_dir='/home/naruto/PycharmProjects/knifey_spoony_demo/data/knifey-spoony/train_data_link/'
test_data_dir='/home/naruto/PycharmProjects/knifey_spoony_demo/data/knifey-spoony/test/'
gen = ImageDataGenerator()
generator_train = gen.flow_from_directory(train_data_dir, target_size=(height, height),batch_size=train_batch_size)
generator_test = gen.flow_from_directory(test_data_dir, target_size=(height, height),batch_size=test_batch_size)

Found 4170 images belonging to 3 classes.
Found 530 images belonging to 3 classes.


In [7]:
# base_model = VGG16(include_top=True, weights='imagenet')需要下载，速度慢，
# include_top=true时下载也比较慢，需要测试等于true时的模型结构
# input_tensor=Input((height, height, 3))报错ValueError: The shape of the input to "Flatten" is not fully defined (got (None, None, 2048).
#  Make sure to pass a complete "input_shape" or "batch_input_shape" argument to the first layer in your model.，是因为此时用的Ｍｏｄｅｌ需要预先指定ｉｎｐｕｔ
base_model = ResNet50(input_tensor=Input((height, height, 3)),weights='imagenet', include_top=False)
base_model.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
input_1 (InputLayer)             (None, 224, 224, 3)   0                                            
____________________________________________________________________________________________________
zero_padding2d_1 (ZeroPadding2D) (None, 230, 230, 3)   0           input_1[0][0]                    
____________________________________________________________________________________________________
conv1 (Conv2D)                   (None, 112, 112, 64)  9472        zero_padding2d_1[0][0]           
____________________________________________________________________________________________________
bn_conv1 (BatchNormalization)    (None, 112, 112, 64)  256         conv1[0][0]                      
___________________________________________________________________________________________

In [8]:
for i ,layer in enumerate(base_model.layers):
    print("layer_{0}:\t{1}\t{2}".format(i,layer.trainable, layer.name))

layer_0:	False	input_1
layer_1:	True	zero_padding2d_1
layer_2:	True	conv1
layer_3:	True	bn_conv1
layer_4:	True	activation_1
layer_5:	True	max_pooling2d_1
layer_6:	True	res2a_branch2a
layer_7:	True	bn2a_branch2a
layer_8:	True	activation_2
layer_9:	True	res2a_branch2b
layer_10:	True	bn2a_branch2b
layer_11:	True	activation_3
layer_12:	True	res2a_branch2c
layer_13:	True	res2a_branch1
layer_14:	True	bn2a_branch2c
layer_15:	True	bn2a_branch1
layer_16:	True	add_1
layer_17:	True	activation_4
layer_18:	True	res2b_branch2a
layer_19:	True	bn2b_branch2a
layer_20:	True	activation_5
layer_21:	True	res2b_branch2b
layer_22:	True	bn2b_branch2b
layer_23:	True	activation_6
layer_24:	True	res2b_branch2c
layer_25:	True	bn2b_branch2c
layer_26:	True	add_2
layer_27:	True	activation_7
layer_28:	True	res2c_branch2a
layer_29:	True	bn2c_branch2a
layer_30:	True	activation_8
layer_31:	True	res2c_branch2b
layer_32:	True	bn2c_branch2b
layer_33:	True	activation_9
layer_34:	True	res2c_branch2c
layer_35:	True	bn2c_branc

no fine-truning

In [11]:
#这个地方比较坑，调试比较费力，按照网上的操作,base_model.output的输出是tensor，以后都是tensor， 传递给其他层，切记，
# TypeError: 'NoneType' object is not callable
# 网上还将func 模型传给sequential（add）会出问题，#ValueError: Variable bn_conv1/moving_mean/biased already exists, disallowed.
# Did you mean to set reuse=True in VarScope? Originally defined at:应该是版本问题，报错说变量存在，设置需要重用
x=Flatten()(base_model.layers[172].output)#不必指定Input,或者base_model的output，就是最后一层
# transfer_layer = model.get_layer('activation_48')
# #获取权重tensor 切记和layer的区别,否则weights维度为none
#【7,7,512】直接FC到3也不错，单独只polling操作也可
# x=GlobalAveragePooling2D()(base_model.layers[172].output)
x = Dropout(0.25)(x)
x = Dense(3, activation='softmax')(x)
model = Model(base_model.input, x)

In [17]:
history_ft = model.fit_generator(
    generator_train,#可自定义
    # samples_per_epoch=4170,  # nb_train_samples，Basically steps_per_epoch = samples_per_epoch/batch_size
    # steps_per_epoch=10,  # nb_train_samples#每轮epoch遍历的samples
    validation_data=generator_test,#可自定义
    nb_epoch=10#,
    # verbose=1#,#控制显示方式，冗长
    # validation_steps=530//64
    # workers=8
    #use_multiprocessing=True,
    # epochs=100
    # nb_val_samples=530 # nb_val_samples`->`validation_steps
)


/home/naruto/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:6: UserWarning: Update your `fit_generator` call to the Keras 2 API: `fit_generator(<keras.pre..., epochs=10, validation_data=<keras.pre...)`
  


TypeError: fit_generator() takes at least 3 arguments (4 given)

fine-truning

In [19]:
for layer in base_model.layers[:172]:
   layer.trainable = False
for layer in base_model.layers[172:]:
   layer.trainable = True
# model.summary()
for i ,layer in enumerate(base_model.layers):
    print("layer_{0}:\t{1}\t{2}".format(i,layer.trainable, layer.name))

layer_0:	False	input_1
layer_1:	False	zero_padding2d_1
layer_2:	False	conv1
layer_3:	False	bn_conv1
layer_4:	False	activation_1
layer_5:	False	max_pooling2d_1
layer_6:	False	res2a_branch2a
layer_7:	False	bn2a_branch2a
layer_8:	False	activation_2
layer_9:	False	res2a_branch2b
layer_10:	False	bn2a_branch2b
layer_11:	False	activation_3
layer_12:	False	res2a_branch2c
layer_13:	False	res2a_branch1
layer_14:	False	bn2a_branch2c
layer_15:	False	bn2a_branch1
layer_16:	False	add_1
layer_17:	False	activation_4
layer_18:	False	res2b_branch2a
layer_19:	False	bn2b_branch2a
layer_20:	False	activation_5
layer_21:	False	res2b_branch2b
layer_22:	False	bn2b_branch2b
layer_23:	False	activation_6
layer_24:	False	res2b_branch2c
layer_25:	False	bn2b_branch2c
layer_26:	False	add_2
layer_27:	False	activation_7
layer_28:	False	res2c_branch2a
layer_29:	False	bn2c_branch2a
layer_30:	False	activation_8
layer_31:	False	res2c_branch2b
layer_32:	False	bn2c_branch2b
layer_33:	False	activation_9
layer_34:	False	res2c_

可以看到最后三层可ｔｒａｉｎａｂｌｅ

In [20]:
model.compile(optimizer='adadelta',
              loss='binary_crossentropy',
              metrics=['accuracy'])

# 节省内存，类似tf的queue
history_ft = model.fit_generator(
    generator_train,#可自定义
    # samples_per_epoch=4170,  # nb_train_samples，Basically steps_per_epoch = samples_per_epoch/batch_size
    # steps_per_epoch=10,  # nb_train_samples#每轮epoch遍历的samples
    validation_data=generator_test,#可自定义
    nb_epoch=10,
    verbose=1,#控制显示方式，冗长
    validation_steps=530//64,
    workers=8,
    # use_multiprocessing=True,
    # epochs=100
    # nb_val_samples=530 # nb_val_samples`->`validation_steps
)

/home/naruto/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:14: UserWarning: Update your `fit_generator` call to the Keras 2 API: `fit_generator(<keras.pre..., verbose=1, workers=8, validation_data=<keras.pre..., epochs=10, validation_steps=8)`
  


TypeError: fit_generator() takes at least 3 arguments (7 given)